In [ ]:
# Import relevant functionality
from langchain_anthropic import ChatAnthropic
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.prebuilt import create_react_agent

# Create the agent
memory = SqliteSaver.from_conn_string(":memory:")
model = ChatAnthropic(model_name="claude-3-sonnet-20240229")
search = TavilySearchResults(max_results=2)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

# Use the agent
config = {"configurable": {"thread_id": "abc123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob! and i live in sf")]}, config
):
    print(chunk)
    print("----")

for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather where I live?")]}, config
):
    print(chunk)
    print("----")

In [ ]:
%pip install -U langchain-community langgraph langchain-anthropic tavily-python

In [1]:
import getpass
import os

os.environ["TAVILY_API_KEY"] = getpass.getpass()

 ········


In [2]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1717674270, 'localtime': '2024-06-06 4:44'}, 'current': {'last_updated_epoch': 1717673400, 'last_updated': '2024-06-06 04:30', 'temp_c': 9.0, 'temp_f': 48.2, 'is_day': 0, 'condition': {'text': 'Mist', 'icon': '//cdn.weatherapi.com/weather/64x64/night/143.png', 'code': 1030}, 'wind_mph': 2.2, 'wind_kph': 3.6, 'wind_degree': 10, 'wind_dir': 'N', 'pressure_mb': 1011.0, 'pressure_in': 29.86, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 100, 'cloud': 0, 'feelslike_c': 7.3, 'feelslike_f': 45.1, 'windchill_c': 10.9, 'windchill_f': 51.6, 'heatindex_c': 12.0, 'heatindex_f': 53.6, 'dewpoint_c': 9.6, 'dewpoint_f': 49.3, 'vis_km': 6.4, 'vis_miles': 3.0, 'uv': 1.0, 'gust_mph': 11.3, 'gust_kph': 18.1}}"}, {'url': 'https://world-weather.info/forecast/usa/san_francisc

In [3]:
import getpass
import os

os.environ["COHERE_API_KEY"] = getpass.getpass()

from langchain_cohere import ChatCohere

model = ChatCohere(model="command-r")

 ········


In [4]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

"Hi! How's it going? I'm here to help you with whatever you need. Do you have any questions or topics you'd like to discuss?"

In [5]:
model_with_tools = model.bind_tools(tools)

In [9]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hi! How's it going? I hope you're having a fantastic day today! 😊
ToolCalls: []


In [10]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'weather in SF'}, 'id': 'df0749cba2da47199db69b640b263b73'}]


In [11]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [12]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', id='a7542f6e-1dbd-42dc-9b6b-f0e955d3236f'),
 AIMessage(content="Hi! How's it going? I hope you're having a fantastic day today! 😊", additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '393a42d0-aaaf-4306-957a-47349a846259', 'token_count': {'input_tokens': 68, 'output_tokens': 19}}, response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '393a42d0-aaaf-4306-957a-47349a846259', 'token_count': {'input_tokens': 68, 'output_tokens': 19}}, id='run-3c665ad4-d66d-4c01-b732-b0c7b3a329ad-0')]

In [13]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
)
response["messages"]

[HumanMessage(content='whats the weather in sf?', id='6eb3e26c-5d65-4e73-8ea1-474a977db7a9'),
 AIMessage(content='', additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '8d1759d4-0595-4155-a6b5-710d2c1e14c9', 'tool_calls': [{'id': '42387d18cb5842d1922f0f67488354e5', 'function': {'name': 'tavily_search_results_json', 'arguments': '{"query": "sf weather"}'}, 'type': 'function'}], 'token_count': {'output_tokens': 15}}, response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '8d1759d4-0595-4155-a6b5-710d2c1e14c9', 'tool_calls': [{'id': '42387d18cb5842d1922f0f67488354e5', 'function': {'name': 'tavily_search_results_json', 'arguments': '{"query": "sf weather"}'}, 'type': 'function'}], 'token_count': {'output_tokens': 15}}, id='run-1ede98d4-e863-4f97-bd21-7b2fcb3de6f3-0', tool_calls=[{'name': 'tavily_sea

In [14]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '6c3fded1-cab5-40c1-a2ac-370be10a52d1', 'tool_calls': [{'id': 'f16de3e665224e0493ba5915abea8b1e', 'function': {'name': 'tavily_search_results_json', 'arguments': '{"query": "sf weather"}'}, 'type': 'function'}], 'token_count': {'output_tokens': 15}}, response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '6c3fded1-cab5-40c1-a2ac-370be10a52d1', 'tool_calls': [{'id': 'f16de3e665224e0493ba5915abea8b1e', 'function': {'name': 'tavily_search_results_json', 'arguments': '{"query": "sf weather"}'}, 'type': 'function'}], 'token_count': {'output_tokens': 15}}, id='run-7eaf6bfa-72d9-4052-af10-324a395e6943-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'sf weather'}, 'id': '9d260f41e5a2

In [15]:
async for event in agent_executor.astream_events(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}, version="v1"
):
    kind = event["event"]
    if kind == "on_chain_start":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print(
                f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
            )
    elif kind == "on_chain_end":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print()
            print("--")
            print(
                f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
            )
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")

C:\Users\scb\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_core\_api\beta_decorator.py:87: LangChainBetaWarning: This API is in beta and may change in the future.
  warn_beta(


--
Starting tool: tavily_search_results_json with inputs: {'query': 'sf weather'}
Done tool: tavily_search_results_json
Tool output was: [{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1717675395, 'localtime': '2024-06-06 5:03'}, 'current': {'last_updated_epoch': 1717675200, 'last_updated': '2024-06-06 05:00', 'temp_c': 9.0, 'temp_f': 48.2, 'is_day': 0, 'condition': {'text': 'Mist', 'icon': '//cdn.weatherapi.com/weather/64x64/night/143.png', 'code': 1030}, 'wind_mph': 2.2, 'wind_kph': 3.6, 'wind_degree': 10, 'wind_dir': 'N', 'pressure_mb': 1012.0, 'pressure_in': 29.87, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 100, 'cloud': 0, 'feelslike_c': 7.4, 'feelslike_f': 45.2, 'windchill_c': 10.8, 'windchill_f': 51.4, 'heatindex_c': 11.8, 'heatindex_f': 53.2, 'dewpoint_c': 9.7, 'dewpoint_f': 49.4, 'vis_km'

NotImplementedError in LogStreamCallbackHandler.on_llm_end callback: NotImplementedError('Trying to load an object that doesn\'t implement serialization: {\'lc\': 1, \'type\': \'not_implemented\', \'id\': [\'cohere\', \'types\', \'chat_citation\', \'ChatCitation\'], \'repr\': "ChatCitation(start=43, end=57, text=\'9.0°C (48.2°F)\', document_ids=[\'tavily_search_results_json:0:0\'])"}')


In [16]:
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")

In [17]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [18]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob!")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="Hi Bob! It's nice to meet you. How's it going today?", additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': 'b45227f7-584c-42aa-b0bd-d308c10d3d61', 'token_count': {'input_tokens': 70, 'output_tokens': 16}}, response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': 'b45227f7-584c-42aa-b0bd-d308c10d3d61', 'token_count': {'input_tokens': 70, 'output_tokens': 16}}, id='run-a05bb497-d539-41b0-8bba-f94fe80297c6-0')]}}
----


In [19]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="Your name is Bob! Did you forget? It's an honor to meet you, Bob.", additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '630fc4c4-e2cf-498e-b257-1bdd8bc55627', 'token_count': {'input_tokens': 96, 'output_tokens': 19}}, response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '630fc4c4-e2cf-498e-b257-1bdd8bc55627', 'token_count': {'input_tokens': 96, 'output_tokens': 19}}, id='run-c2c87569-9f81-4054-a014-efaaafccc3cf-0')]}}
----


In [20]:
config = {"configurable": {"thread_id": "xyz123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="I'm sorry, but as an AI chatbot, I don't know your name. If you tell me your name, I will be happy to address you by it.", additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': 'e710990a-b236-4d42-8b9c-d956d667eed3', 'token_count': {'input_tokens': 70, 'output_tokens': 34}}, response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': 'e710990a-b236-4d42-8b9c-d956d667eed3', 'token_count': {'input_tokens': 70, 'output_tokens': 34}}, id='run-9d6ec4f8-22cd-4749-893a-157b3d1e394d-0')]}}
----
